In [99]:
import pandas as pd
import numpy as np
import torch
import sklearn

In [100]:
torch.manual_seed(1)

In [101]:
train_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data_sample/2021-ai-w4-p2/train.csv")
test_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data_sample/2021-ai-w4-p2/test.csv")
submit_df = pd.read_csv("drive/MyDrive/Colab Notebooks/data_sample/2021-ai-w4-p2/sample.csv")

In [102]:
print(train_df.head())
print(test_df.head())
print(submit_df.head())

    N    P    K  temperature   humidity        ph    rainfall        label
0  22   36   16    30.581395  50.771481  8.184229   64.585596    mothbeans
1  18   27   41    22.365094  92.308824  7.175344  104.821633  pomegranate
2  61   68   50    35.214628  91.497251  6.793245  243.074507       papaya
3  26   72   22    28.767949  37.577921  4.674942   91.720849   pigeonpeas
4   2  140  197    22.697801  92.822234  5.534567  105.050823        apple
     N    P    K  temperature   humidity        ph    rainfall
0  101   17   47    29.494014  94.729813  6.185053   26.308209
1   98    8   51    26.179346  86.522581  6.259336   49.430510
2   59   62   49    43.360515  93.351916  6.941497  114.778071
3   44   60   55    34.280461  90.555616  6.825371   98.540477
4   30  137  200    22.914300  90.704756  5.603413  118.604465
   id   label
0   0  coffee
1   1  coffee
2   2  coffee
3   3  coffee
4   4  coffee


In [103]:
# preprocessing
from sklearn.preprocessing import LabelEncoder,StandardScaler
le = LabelEncoder()

In [104]:
le.fit(train_df['label'])
train_df['label'] = le.transform(train_df['label'])

In [105]:
scaler = StandardScaler()

In [106]:
x_train = np.array(train_df.drop(['label'], axis= 1))
y_train = np.array(train_df['label'])
x_test = np.array(test_df)

In [107]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_train)

In [108]:
x_train = torch.Tensor(x_train).cuda()
y_train = torch.LongTensor(y_train).cuda()
x_test = torch.Tensor(x_test).cuda()

In [109]:
print(x_train.shape,x_test.shape,y_train.shape)
print(y_train)

torch.Size([1650, 7]) torch.Size([1650, 7]) torch.Size([1650])
tensor([13, 19, 17,  ..., 12,  7, 10], device='cuda:0')


In [110]:
class_num = len(le.classes_)
w = torch.zeros((7,class_num),requires_grad=True,device="cuda")
b = torch.zeros(1,requires_grad=True,device="cuda")

optim = torch.optim.SGD([w,b],lr = 1e-2)

loss = torch.nn.CrossEntropyLoss()

epochs =100000

In [111]:
for epoch in range(epochs+1):
  optim.zero_grad()

  h = x_train.matmul(w)+b
  cost = loss(h,y_train)

  cost.backward()
  optim.step()
  if epoch % 5000 == 0:
    print(cost.item())

3.091043710708618
0.859039843082428
0.6116042733192444
0.5020124912261963
0.43615052103996277
0.3908868730068207
0.35735565423965454
0.33129021525382996
0.3103301525115967
0.29304301738739014
0.27849939465522766
0.26606449484825134
0.25529003143310547
0.2458471804857254
0.23749037086963654
0.23003274202346802
0.22332757711410522
0.21726049482822418
0.21173687279224396
0.2066822052001953
0.20203502476215363


In [112]:
with torch.no_grad():
  predict = torch.softmax(x_test.matmul(w)+b,dim =1)
  predict = torch.argmax(predict,dim =1)
  predict = le.inverse_transform(predict.cpu().detach().numpy())